# Sprint13

In [1]:
import numpy as np
import pandas as pd
import inspect, os

# ログ表示
class Log:
    # 表示ログレベル
    Loglevel = 0
    def logLevelset(level):
        # 表示ログレベルの設定
        Log.Loglevel = level
    def log_print(level, std, variable=None, obj=None):
        if(Log.Loglevel >= level):
            if(obj is not None):
                print('class:{}'.format(obj.__class__.__name__), end=' ')
            frame = inspect.currentframe().f_back
            print("method:{} line:{}"
                .format(frame.f_code.co_name, frame.f_lineno))
            if(variable is None):
                print(std)
            else:
                if(len(variable) == 1):
                    print(std.format(variable))
                else:
                    print(std.format(*variable))

In [2]:
Log.logLevelset(2)

# 2D convolution

In [8]:
class Conv2d:
    def __init__(self, FN, FH, FW, padding, ir, stride=1):
        self.FN = FN # フィルタ数
        self.FH = FH # フィルタの高さ
        self.FW = FW # フィルタの幅
        self.P = padding # パディング数
        self.S = stride # スライド数
        self.B = 1
        self.W = np.arange(FN*FH*FW).reshape(FN, FH, FW).astype(np.float)
        self.forward_x = None
        self.ir = ir
    
    def forward(self, X):
        
        self.forward_x = X.copy()
        
        # 各パラメータを設定
        # 入力データの次元数を取得
        n_channel, n_high, n_wide = X.shape
        
        # パディング対応
        X = np.pad(X, ((0,0), (self.P,self.P), (self.P,self.P)), 'constant')
        
        indexs = np.arange(n_wide)
        out_c = self.FN
        out_h = np.trunc( (n_high + 2*self.P - self.FH )/self.S +1).astype(np.int)
        out_w = np.trunc( (n_wide + 2*self.P - self.FW )/self.S +1 ).astype(np.int)

        output = np.empty((out_c, out_h, out_w))
        
        for FN in range(out_c):
            # 高さ×幅分回す
            j = 0
            for high in range(0, n_high, self.S):
                i = 0
                for wide in range(0, n_wide, self.S):
                    # 幅方向のインデックスを取得
                    index = indexs[wide:wide+self.FW]
                    # 高さ方向にインデックスをブロードキャスト
                    output[FN, j, i] = np.sum(X[:,high:high+self.FH, index]*self.W[FN,:,:]) + self.B

                    # 次の幅方向にスライドできるかチェック
                    if((wide + self.FW + self.S -1) >= n_wide):
                        break

                # 次の高さ方向にスライドできるかチェック
                if((high + self.FH + self.S -1) >= n_high):
                    break
        
        return output
    
    def backward(self, delta):
        """
        バイアスの逆伝搬
        """
        # 勾配
        dB = np.mean(delta, axis=(1,2))

        #Log.log_print(1, "dB.shape:{}", (dB.shape), self)
        #Log.log_print(2, "dB:{}", dB, self)

        # 更新
        self.B -= self.ir*dB.astype(np.float)

        #Log.log_print(1, "self.B.shape:{}", (self.B.shape), self)
        #Log.log_print(2, "self.B:{}", self.B , self)
        
        """
        出力の逆伝搬
        """
        delta = delta.astype(np.float)
        
        #　勾配
        n_channel, n_high, n_wide = self.forward_x.shape
        
        dX = np.empty((n_channel, n_high, n_wide)).astype(np.float)
        
        # InPut のチャネル数
        for channel in range(n_channel):
            temp = 0
            for high in range(n_high):
                for wide in range(n_wide):
                    for j in range(self.W.shape[0]):
                        for i in range(self.W.shape[1]):
                            if( ((high - j) < 0) or ((high - j) >= delta.shape[0]) ):
                                a = 0
                            elif( ((wide - i) < 0) or ( (wide - i) >= delta.shape[1])):
                                a = 0
                            else:
                                a = delta[:, high - j, wide - i]
                            temp += a * self.W[:, j, i]
                    dX[channel, high, wide] = np.sum(temp)

        """
        カーネルの逆伝搬
        """
        # 勾配
        n_FN, n_high, n_wide = self.W.shape
        n_out_channel, n_out_wide, n_out_high = delta.shape
        
        indexs = np.arange(self.forward_x.shape[1])
        dW = np.empty((n_FN, n_high, n_wide)).astype(np.float)
        # FNの数
        for FN in range(n_FN):
            j = 0
            for high in range(n_high):
                i = 0
                for wide in range(n_wide):
                    index = indexs[wide:wide+n_out_wide]
                    dW[FN, j, i] = np.sum( np.dot(self.forward_x[:, high:high+n_out_high, index], delta[FN, :, :]) )
                i += 1
            j += 1
        
        # 更新
        self.W -= self.ir*dW

        return dX 

# 実行結果

In [9]:
test = np.arange(8*8).reshape(1,8,8).astype(np.float)
test

array([[[ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.],
        [ 8.,  9., 10., 11., 12., 13., 14., 15.],
        [16., 17., 18., 19., 20., 21., 22., 23.],
        [24., 25., 26., 27., 28., 29., 30., 31.],
        [32., 33., 34., 35., 36., 37., 38., 39.],
        [40., 41., 42., 43., 44., 45., 46., 47.],
        [48., 49., 50., 51., 52., 53., 54., 55.],
        [56., 57., 58., 59., 60., 61., 62., 63.]]])

In [10]:
instance = Conv2d(2, 3, 3, 0, 0.01)

In [11]:
test2 = instance.forward(test)
test2

array([[[2.09500000e+003, 6.01346953e-154, 6.13292705e-290,
         9.42526130e+171, 9.50611371e-007, 6.01346954e-154],
        [2.17234203e-153, 4.80549070e+116, 2.68993450e-110,
         3.60705816e-085, 1.76004505e+078, 2.41317562e+185],
        [3.60705816e-085, 3.62977568e+228, 1.95057883e-110,
         6.01347002e-154, 6.01347002e-154, 1.70575323e+248],
        [9.16526748e+242, 1.01345567e+267, 6.01334504e-154,
         9.82157873e+252, 1.44420343e+214, 6.01334586e-154],
        [9.82157873e+252, 5.28595592e-085, 2.05037367e-115,
         5.86060639e-101, 3.60705816e-085, 2.41317558e+185],
        [4.97532656e+180, 2.85557116e-110, 4.35294653e-114,
         1.03472182e-259, 6.01347002e-154, 2.17237759e-153]],

       [[6.46900000e+003, 5.03887214e+175, 4.97029684e+180,
         3.53447545e+246, 1.35896403e-110, 5.72241203e-153],
        [8.88017290e+252, 4.55072007e+198, 6.01334434e-154,
         9.82157873e+252, 3.74168445e+233, 3.10573437e-115],
        [1.91570149e+227, 4.67

In [12]:
instance.backward(test2)

array([[[5.82210000e+004, 4.53498493e+176, 4.47336290e+181,
         3.18102790e+247, 6.71550335e+247, 1.06034263e+248,
         1.06034263e+248, 1.06034263e+248],
        [7.99216621e+253, 1.68723391e+254, 2.66405293e+254,
         3.54799544e+254, 4.53015377e+254, 5.61052793e+254,
         5.61052793e+254, 5.61052793e+254],
        [6.67614868e+254, 7.83057115e+254, 9.07379536e+254,
         1.02523848e+255, 1.15291900e+255, 1.29042111e+255,
         1.29042111e+255, 1.29042111e+255],
        [1.29042111e+255, 1.29042111e+255, 1.29042111e+255,
         1.29042111e+255, 1.29042111e+255, 1.29042111e+255,
         1.29042111e+255, 1.29042111e+255],
        [1.29042111e+255, 1.29042111e+255, 1.29042111e+255,
         1.29042111e+255, 1.29042111e+255, 1.29042111e+255,
         1.29042111e+255, 1.29042111e+255],
        [1.29042111e+255, 1.29042111e+255, 1.29042111e+255,
         1.29042111e+255, 1.29042111e+255, 1.29042111e+255,
         1.29042111e+255, 1.29042111e+255],
        [1.29042

# Max Pooling

In [13]:
class MaxPool2D:
    def __init__(self, FH, FW, padding=0, stride=None):
        self.FH = FH # フィルタの高さ
        self.FW = FW # フィルタの幅
        self.P = padding # パディング数
        
        if (stride is None):
            self.S = FW # スライド数
        else:
            self.S = stride
        self.forward_index = None
    
    def forward(self, X):
          
        # 各パラメータを設定
        # 入力データの次元数を取得
        n_channel, n_high, n_wide = X.shape
        
        self.forward_index = np.zeros(X.shape)
        
        # パディング対応
        X = np.pad(X, ((0,0), (self.P,self.P), (self.P,self.P)), 'constant')
        
        indexs = np.arange(n_wide)
        out_c = n_channel
        out_h = np.trunc( (n_high + 2*self.P - self.FH)/self.S +1 ).astype(np.int)
        out_w = np.trunc( (n_wide + 2*self.P - self.FW)/self.S +1 ).astype(np.int)

        output = np.empty((out_c, out_h, out_w))
        
        for FN in range(out_c):
            # 高さ×幅分回す
            j = 0
            for high in range(0, n_high, self.S):
                i = 0
                for wide in range(0, n_wide, self.S):
                    # 幅方向のインデックスを取得
                    index = indexs[wide:wide+self.FW]
                    # 高さ方向にインデックスをブロードキャスト
                    # 最大値のインデックスを取得
                    max_index = np.argmax( X[FN,high:high+self.FH, index] )
                    max_index = np.unravel_index(max_index, (self.FH, self.FW))
                    self.forward_index[FN, max_index[0]+high, max_index[1]+wide] = 1
                    
                    output[FN, j, i] = np.max( X[FN,high:high+self.FH, index] )
                    
                    i += 1
                    # 次の幅方向にスライドできるかチェック
                    if((wide + self.FW + self.S -1) >= n_wide):
                        break
                j += 1
                # 次の高さ方向にスライドできるかチェック
                if((high + self.FH + self.S -1) >= n_high):
                    break
        
        return output
    
    def backward(self, delta):
        
        n_channel, n_high, n_wide = delta.shape
        filte = np.ones((n_channel,self.FH, self.FW))
        
        result_high = []
        for high in range(n_high):
            result_wide = []
            for wide in range(n_wide):
                result_wide.append( filte*delta[:,high,wide] )
            result_high.append( np.hstack(result_wide) )
        result_Filter = np.vstack(result_high).T
        
        return result_Filter*self.forward_index

# 実行結果

In [14]:
Maxpool = np.arange(2*8*8).reshape(2,8,8).astype(np.float)
Maxpool

array([[[  0.,   1.,   2.,   3.,   4.,   5.,   6.,   7.],
        [  8.,   9.,  10.,  11.,  12.,  13.,  14.,  15.],
        [ 16.,  17.,  18.,  19.,  20.,  21.,  22.,  23.],
        [ 24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.],
        [ 32.,  33.,  34.,  35.,  36.,  37.,  38.,  39.],
        [ 40.,  41.,  42.,  43.,  44.,  45.,  46.,  47.],
        [ 48.,  49.,  50.,  51.,  52.,  53.,  54.,  55.],
        [ 56.,  57.,  58.,  59.,  60.,  61.,  62.,  63.]],

       [[ 64.,  65.,  66.,  67.,  68.,  69.,  70.,  71.],
        [ 72.,  73.,  74.,  75.,  76.,  77.,  78.,  79.],
        [ 80.,  81.,  82.,  83.,  84.,  85.,  86.,  87.],
        [ 88.,  89.,  90.,  91.,  92.,  93.,  94.,  95.],
        [ 96.,  97.,  98.,  99., 100., 101., 102., 103.],
        [104., 105., 106., 107., 108., 109., 110., 111.],
        [112., 113., 114., 115., 116., 117., 118., 119.],
        [120., 121., 122., 123., 124., 125., 126., 127.]]])

In [54]:
instance = MaxPool2D(2,2)
Maxpool_result = instance.forward(Maxpool)
Maxpool_result

array([[[  9.,  11.,  13.,  15.],
        [ 25.,  27.,  29.,  31.],
        [ 41.,  43.,  45.,  47.],
        [ 57.,  59.,  61.,  63.]],

       [[ 73.,  75.,  77.,  79.],
        [ 89.,  91.,  93.,  95.],
        [105., 107., 109., 111.],
        [121., 123., 125., 127.]]])

In [55]:
instance.backward(Maxpool_result)

array([[[  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
        [  0.,   9.,   0.,  25.,   0.,  41.,   0.,  57.],
        [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
        [  0.,  11.,   0.,  27.,   0.,  43.,   0.,  59.],
        [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
        [  0.,  13.,   0.,  29.,   0.,  45.,   0.,  61.],
        [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
        [  0.,  15.,   0.,  31.,   0.,  47.,   0.,  63.]],

       [[  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
        [  0.,  73.,   0.,  89.,   0., 105.,   0., 121.],
        [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
        [  0.,  75.,   0.,  91.,   0., 107.,   0., 123.],
        [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
        [  0.,  77.,   0.,  93.,   0., 109.,   0., 125.],
        [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
        [  0.,  79.,   0.,  95.,   0., 111.,   0., 127.]]])

# 説明文
## Convolution層
畳み込み層とプーリング層では下図のように入力のニューロンの一部の領域を絞って、局所的に次の層へと対応付けをしていく。各層はフィルタと呼ばれる検出器をいくつも持っているイメージになる。
![](https://deepage.net/img/convolutional_neural_network/conv.jpg)
画像認識の例でいうと、最初の層でエッジを検出して、次の層でテクスチャを検出し、さらに次のそうではより抽象的な猫の耳などの特徴を検出する。CNNはこういった特徴を抽出するための検出器であるフィルタのパラメータを自動で学習していく。<br>
    Convolution層は空間的な情報を維持することができる。widthとheightとdepthの3次元を入力値として3次元を出力する例を考えてみる。
![](https://deepage.net/img/convolutional_neural_network/conv_volume.jpg)
$Convolution$層では、各フィルターごとに畳み込み演算された結果が出力される。$RGB$チャンネルの入力画像の場合、フィルタも$3$次元となり$3×3×3$のフィルタであれば、合計で$27$個の重みフィルタとなる。上図のように、フィルタごとに畳み込み演算した出力が次の層の$1$つのユニットとなる。そして出力はフィルターの数の深さを持つことになる。<br>

Fully Connected層と同様に、フィルタごとに内積を計算しバイアスを足して活性化関数を適用する。重みフィルタのパラメータが$w_{1}...w_{27}$で対象の局所領域が$x_{1}...x_{27}$
で対象の局所領域が だった場合、出力のユニット は$y$は
$$y=relu(\sum_{i=1}^{27}w_{i}x_{i}+b)$$
となる。$relu()$はReLU(Rectified Linear Unit)のことで、$max(0, x)$のような実装をする。Convolution層ではこの活性化関数を使うことが多い。

## Pooling層
Pooling層はたいてい、Convolutoin層の後に適用される。入力データをより扱いやすい形に変形するために、情報を圧縮し、down samplingする。<br>

max poolingと呼ばれる手法では、操作は以下のように小領域に対して、最大のものを選択する操作となる。
![](https://deepage.net/img/convolutional_neural_network/max_pooling.jpg)
このように情報を圧縮することで

- 微小な位置変化に対して頑健となる
- ある程度過学習を抑制する
- 計算コストを下げる

といった効果があり、Convolution層とPooling層で特徴を検出する働きをする。

# CNNは全結合層のみのニューラルネットワークとどのように違うか


CNNでは、隠れ層は「畳み込み層」と「プーリング層」で構成されている。畳み込み層は、前の層で近くにあるノードにフィルタ処理して「特徴マップ」を得る。プーリング層は、畳込み層から出力された特徴マップを、さらに縮小して新たな特徴マップとする。この際に着目する領域のどの値を用いるかは、最大値を得ることで、画像の多少のずれも吸収される。したがって、この処理により画像の位置移動に対する普遍性を獲得したことになる。<br>

畳み込み層は画像の局所的な特徴を抽出し、プーリング層は局所的な特徴をまとめあげる処理をしている。つまり、これらの処理の意味するところは、入力画像の特徴を維持しながら画像を縮小処理していることにななる。今までの画像縮小処理と異なるところは、画像の特徴を維持しながら画像の持つ情報量を大幅に圧縮できるところである。これを言い換えると、画像の「抽象化」とも言える。これは画期的なことである。ネットワークに記憶された、この抽象化された画像イメージを用いて、入力される画像を認識、つまり画像の分類をすることができるのである。

## 出力サイズとパラメータ数の計算

1.

- 入力サイズ : 144×144, 3チャンネル
- フィルタサイズ : 3×3, 6チャンネル
- ストライド : 1
- パディング : なし

$N_{h,out} =  \frac{N_{h,in}+2P_{h}-F_{h}}{S_{h}} + 1\\
N_{h,out} =  \frac{144+2×0-3}{1} + 1\\
N_{h,out} =  142\\
N_{w,out} =  \frac{N_{w,in}+2P_{w}-F_{w}}{S_{w}} + 1
N_{w,out} =  \frac{144+2×0-3}{1} + 1\\
N_{w,out} =  142$

出力サイズ：142×142、6チャンネル<br>
パラメータ：
- フィルタ 3×3×6=54
- ストライド: 1
- パディング: 0
- 計：55

2.

- 入力サイズ : 60×60, 24チャンネル
-    フィルタサイズ : 3×3, 48チャンネル
-    ストライド　: 1
-    パディング : なし

$N_{h,out} =  \frac{N_{h,in}+2P_{h}-F_{h}}{S_{h}} + 1\\
N_{h,out} =  \frac{60+2×0-3}{1} + 1\\
N_{h,out} =  58\\
N_{w,out} =  \frac{N_{w,in}+2P_{w}-F_{w}}{S_{w}} + 1
N_{w,out} =  \frac{60+2×0-3}{1} + 1\\
N_{w,out} =  58$

出力サイズ：58×58、48チャンネル<br>
パラメータ：
- フィルタ 3×3×48=432
- ストライド: 1
- パディング: 0
- 計：433

3.

- 入力サイズ : 20×20, 10チャンネル
-    フィルタサイズ : 3×3, 20チャンネル
-    ストライド　: 2
-    パディング : なし

$N_{h,out} =  \frac{N_{h,in}+2P_{h}-F_{h}}{S_{h}} + 1\\
N_{h,out} =  \frac{20+2×0-3}{2} + 1\\
N_{h,out} =  9\\
N_{w,out} =  \frac{N_{w,in}+2P_{w}-F_{w}}{S_{w}} + 1
N_{w,out} =  \frac{20+2×0-3}{2} + 1\\
N_{w,out} =  9$

出力サイズ：58×58、48チャンネル<br>
パラメータ：
- フィルタ 3×3×20=180
- ストライド: 1
- パディング: 0
- 計：181

## フィルタサイズ